In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axis as axis

from mpl_toolkits.mplot3d import axes3d
import seaborn as sns

from sklearn.preprocessing import scale
import sklearn.linear_model as skl_lm
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import statsmodels.formula.api as smf

sns.set(font_scale=1.25)
plt.style.use('seaborn-white')

<ipython-input-1-128f50fdf07e>:18: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')


In [ ]:
from traitlets.traitlets import Float
url1 = '/content/sample_data/data/PNL.csv'
url2 = '/content/sample_data/data/STD.csv'
url3 = '/content/sample_data/data/Vol.csv'
url4 = '/content/sample_data/data/PnV.csv'
pnl_df = pd.read_csv(url1).drop(["Unnamed: 0", "PNL_normalized"], axis=1)
pnl_df.rename(columns={'PNL-mean': 'PNL'}, inplace=True)

data_df = pd.read_csv(url2).drop(["Unnamed: 0"], axis=1)
data_df['STD_Diff'] =   data_df['STD_Sum'] - data_df['STD_Total']

vol_df = pd.read_csv(url3)
vol_df.rename(columns={'Open': 'Open_Vol'}, inplace=True)
vol_df.rename(columns={'Change %': 'Perc_Change_Vol'}, inplace=True)
vol_df['Perc_Change_Vol'] = vol_df['Perc_Change_Vol'].str.rstrip('%').astype(float)
vol_df['Perc_Change_Vol'] = vol_df['Perc_Change_Vol'].abs()
vol_df.drop(['Price', 'High','Low'], axis=1, inplace=True)

pnv_df = pd.read_csv(url4)
pnv_df['Range'] = (pnv_df['High'] - pnv_df['Low']).abs()
pnv_df.drop(columns=['High', 'Low', 'Last', 'Open','Change'], inplace=True)
pnv_df.rename(columns={'%Chg': 'Perc_Change_Price'}, inplace=True)
pnv_df.rename(columns={'Open Int': 'Open_Int'}, inplace=True)
pnv_df['Perc_Change_Price'] = pnv_df['Perc_Change_Price'].str.rstrip('%').astype(float)
pnv_df['Perc_Change_Price'] = pnv_df['Perc_Change_Price'].abs()

pnv_df

,Date,Perc_Change_Price,Volume,Open_Int,Range
0,2023-05-01,1.46,443900,1886091,2.16
1,2023-04-28,2.70,704355,1881769,2.99
2,2023-04-27,0.62,784946,1880445,1.25
3,2023-04-26,3.59,1084446,1862150,3.88
4,2023-04-25,2.15,784520,1826195,2.57
...,...,...,...,...,...
328,2022-01-07,0.36,979709,1918425,1.03
329,2022-01-06,0.78,1170432,1913391,1.99
330,2022-01-05,0.06,989697,1901975,1.22
331,2022-01-04,1.38,852063,1887131,1.37


Combine the 4 dataframes and export to csv 

In [ ]:
df = pnl_df.merge(data_df, left_on='Date', right_on='Date', how="outer")
df = df.merge(vol_df, left_on='Date', right_on='Date', how="outer")
df = df.merge(pnv_df, left_on='Date', right_on='Date', how="outer")
df.columns
df.describe()
df.dropna(how='any', inplace=True)



Index(['Date', 'PNL', 'STD_A', 'STD_B', 'STD_C', 'STD_D', 'STD_E', 'STD_F',
       'STD_G', 'STD_Sum', 'STD_Total', 'STD_Diff', 'Open_Vol',
       'Perc_Change_Vol', 'Perc_Change_Price', 'Volume', 'Open_Int', 'Range'],
      dtype='object')

,PNL,STD_A,STD_B,STD_C,STD_D,STD_E,STD_F,STD_G,STD_Sum,STD_Total,STD_Diff,Open_Vol,Perc_Change_Vol,Perc_Change_Price,Volume,Open_Int,Range
count,3.420000e+02,342.000000,3.420000e+02,342.000000,342.000000,342.000000,342.000000,342.000000,3.420000e+02,3.420000e+02,342.000000,338.000000,338.000000,333.000000,3.330000e+02,3.330000e+02,333.000000
mean,-2.309942e-01,93869.043750,3.956321e+04,8264.762150,2539.918919,20520.192692,9455.791206,165508.432482,3.397214e+05,2.314130e+05,108308.308766,48.487160,3.386095,1.736667,8.161039e+05,1.698217e+06,2.853934
std,2.687050e+05,37827.831673,9.591631e+04,8054.742040,4433.758801,8127.503352,8429.786836,87642.915202,1.279189e+05,1.037418e+05,35164.003986,8.182379,3.189119,1.385590,2.505062e+05,2.015544e+05,1.361234
min,-7.773490e+05,28827.721692,8.131632e+03,231.173271,0.000000,298.230507,875.540100,34309.683625,1.252099e+05,7.486871e+04,43848.953084,32.810000,0.000000,0.000000,3.969590e+05,1.394021e+06,0.880000
25%,-1.809132e+05,68406.863550,1.839071e+04,3490.725302,598.235416,14984.417255,3744.636144,92163.005483,2.522003e+05,1.626683e+05,83966.705178,42.770000,1.292500,0.780000,6.666300e+05,1.499498e+06,2.060000
50%,-8.675500e+03,84772.328560,2.208326e+04,5843.187238,1216.636520,21344.085798,6673.107945,123813.005550,3.267238e+05,2.223562e+05,102878.054795,48.615000,2.660000,1.420000,7.707760e+05,1.727970e+06,2.670000
75%,1.457685e+05,107858.160316,2.746162e+04,8934.970391,2656.240550,24865.962814,12253.716705,245364.040989,4.059978e+05,2.779595e+05,129432.262713,53.150000,4.327500,2.410000,9.112430e+05,1.810295e+06,3.360000
max,1.251707e+06,243136.848966,1.268140e+06,41889.026330,47690.123423,39292.703565,65505.993487,393959.791481,1.485312e+06,1.276651e+06,301763.370589,78.910000,24.340000,9.570000,2.143563e+06,2.175221e+06,14.870000


create a bianary variable for STD_Ration where 0 if the ratio is below the mean and 1 if it is above the mean STD

In [ ]:
median_diff = df['STD_Diff'].median()
median_diff
bins = [0, median_diff, df['STD_Diff'].max()]

labels = [0, 1]
df['High_Diff'] = pd.cut(df['STD_Diff'], bins = bins, labels = labels)

102210.44159281891

Same for open Vol

In [ ]:
median_vol = df['Open_Vol'].median()
median_vol
bins = [0, median_vol, df['Open_Vol'].max()]

labels = [0, 1]
df['High_Vol'] = pd.cut(df['Open_Vol'], bins = bins, labels = labels)
# df.drop("Open_Vol", axis=1)


48.625

In [ ]:
df.Perc_Change_Price.describe()

count    332.000000
mean       1.739880
std        1.386439
min        0.000000
25%        0.780000
50%        1.420000
75%        2.412500
max        9.570000
Name: Perc_Change_Price, dtype: float64

In [ ]:
median_PC_Price = df['Perc_Change_Price'].median()
median_PC_Price

bins = [0, median_PC_Price, df['Perc_Change_Price'].max()]

labels = [0, 1]
df['High_PC_Price'] = pd.cut(df['Perc_Change_Price'], bins = bins, labels = labels)
# df.drop("Perc_Change_Price", axis=1)

1.42

In [ ]:
median_PC_vol = df['Perc_Change_Vol'].median()

bins = [0, median_PC_vol, df['Perc_Change_Vol'].max()]

labels = [0, 1]
df['High_PC_Vol'] = pd.cut(df['Perc_Change_Vol'], bins = bins, labels = labels)
# df.drop("Perc_Change_Vol", axis=1, inplace=True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/dataframe1.csv'
df.to_csv(file_path, index=False)

Mounted at /content/drive
